In [1]:
# Check P2M, M2P and M2M
import numpy as np
from test_pcm_fmm import pcm_fmm
p = 10
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
vscalesm = vscales[:(p+1)*(p+1)]
src = np.array([0.5, 0.4, 0.3], dtype=np.float64)
dst = np.array([1, 1.1, 2.2], dtype=np.float64)
p2p = np.linalg.norm(src-dst)**-1
sph1_c = np.zeros(3)
sph1_r = 1
sph1_coef = pcm_fmm.fmm_p2m(src-sph1_c, sph1_r, p, vscalesm)
m2p = pcm_fmm.fmm_m2p(dst-sph1_c, sph1_r, p, vscalesm, sph1_coef)
print("|| P2M + M2P - P2P ||  /  || P2P || =", abs((m2p-p2p)/p2p))
sph2_c = -np.ones(3)
sph2_r = sph1_r + np.linalg.norm(sph2_c-sph1_c)
sph2_coef = pcm_fmm.fmm_p2m(src-sph2_c, sph2_r, p, vscalesm)
sph2_coef_m2m = np.zeros_like(sph2_coef)
pcm_fmm.fmm_m2m_baseline(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_m2m)
print("|| P2M(1) + M(1)2M(2) - P2M(2) ||  /  || P2M(2) || =",
      np.linalg.norm(sph2_coef_m2m-sph2_coef) / np.linalg.norm(sph2_coef))
sph3_c = np.array([0, 0, -1])
sph3_r = sph1_r + np.linalg.norm(sph3_c-sph1_c)
sph3_coef = pcm_fmm.fmm_p2m(src-sph3_c, sph3_r, p, vscalesm)
sph3_coef_m2m = np.zeros_like(sph3_coef)
pcm_fmm.fmm_m2m_ztranslate((sph1_c-sph3_c)[2], sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_m2m)
print("|| P2M(1) + M(1)TZ2M(3) - P2M(3) ||  /  || P2M(3) || =",
      np.linalg.norm(sph3_coef_m2m-sph3_coef) / np.linalg.norm(sph3_coef))
sph4_c = sph1_c
sph4_r = 3*sph1_r
sph4_coef = pcm_fmm.fmm_p2m(src-sph4_c, sph4_r, p, vscalesm)
sph4_coef_m2m = np.zeros_like(sph4_coef)
pcm_fmm.fmm_m2m_baseline(sph1_c-sph4_c, sph1_r, sph4_r, p, vscalesm, sph1_coef, sph4_coef_m2m)
print("|| P2M(1) + M(1)2M(4) - P2M(4) ||  /  || P2M(4) || =",
      np.linalg.norm(sph4_coef_m2m-sph4_coef) / np.linalg.norm(sph4_coef))

|| P2M + M2P - P2P ||  /  || P2P || = 1.2811025058542572e-07
|| P2M(1) + M(1)2M(2) - P2M(2) ||  /  || P2M(2) || = 5.294215918618704e-16
|| P2M(1) + M(1)TZ2M(3) - P2M(3) ||  /  || P2M(3) || = 3.419931021185157e-16
|| P2M(1) + M(1)2M(4) - P2M(4) ||  /  || P2M(4) || = 1.0190874152954408e-16


In [1]:
# Check M2L, L2P and L2L
import numpy as np
from test_pcm_fmm import pcm_fmm
pl = 10
pm = 10
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(pm, pl, ngrid)
src = np.array([0.5, 0.4, 0.3], dtype=np.float64)
dst = np.array([1, 1.1, 2.2], dtype=np.float64)
p2p = np.linalg.norm(src-dst)**-1
# Compute multipole coefficients
sph1_c = np.zeros(3)
sph1_r = 2
sph1_coef = pcm_fmm.fmm_p2m(src-sph1_c, sph1_r, pm, vscales[:(pm+1)**2])
# Compute local expansion
sph2_c = np.array([1, 1, 2], dtype=np.float64)
sph2_r = 1.3
sph2_coef = np.zeros((pl+1)*(pl+1))
pcm_fmm.fmm_m2l_baseline(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef)
l2p = pcm_fmm.fmm_l2p(dst-sph2_c, sph2_r, pl, vscales[:(pl+1)**2], sph2_coef)
print("|| P2M + M2L + L2P - P2P ||  /  || P2P || =", abs((l2p-p2p)/p2p))
sph3_c = np.array([1, 1, 2], dtype=np.float64)
sph3_r = 0.6
sph3_coef = np.zeros_like(sph2_coef)
pcm_fmm.fmm_l2l_baseline(sph2_c-sph3_c, sph2_r, sph3_r, pl, vscales[:(pl+1)**2], sph2_coef, sph3_coef)
l2l2p = pcm_fmm.fmm_l2p(dst-sph3_c, sph3_r, pl, vscales[:(pl+1)**2], sph3_coef)
print("|| L(1)2L(2) + L(2)2P - L(1)2P ||  /  || L(1)2P || =", abs((l2l2p-l2p)/l2p))
sph3_c = np.array([1.1, 1.1, 2.1], dtype=np.float64)
sph3_r = 0.6
sph3_coef = np.zeros_like(sph2_coef)
pcm_fmm.fmm_l2l_baseline(sph2_c-sph3_c, sph2_r, sph3_r, pl, vscales[:(pl+1)**2], sph2_coef, sph3_coef)
l2l2p = pcm_fmm.fmm_l2p(dst-sph3_c, sph3_r, pl, vscales[:(pl+1)**2], sph3_coef)
print("|| L(1)2L(3) + L(3)2P - L(1)2P ||  /  || L(1)2P || =", abs((l2l2p-l2p)/l2p))

|| P2M + M2L + L2P - P2P ||  /  || P2P || = 0.500000064067947
|| L(1)2L(2) + L(2)2P - L(1)2P ||  /  || L(1)2P || = 1.157777001331601e-16
|| L(1)2L(3) + L(3)2P - L(1)2P ||  /  || L(1)2P || = 6.946662007989605e-16


In [3]:
# Check optimized M2M for spheres
import numpy as np
from test_pcm_fmm import pcm_fmm
p = 70
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
vscalesm = vscales[:(p+1)*(p+1)]
sph1_c = np.zeros(3)
sph1_r = 1
sph1_coef = np.ones_like(vscalesm)
sph2_c = np.array([0, 0, 1], dtype=np.float64)
sph2_r = sph1_r + np.linalg.norm(sph2_c-sph1_c)
sph3_c = np.array([-1, -2, -10], dtype=np.float64)
sph3_r = sph1_r + np.linalg.norm(sph3_c-sph1_c)
sph2_coef_baseline = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_baseline(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_baseline)
sph3_coef_baseline = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_baseline(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_baseline)
sph2_coef_reflection = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_reflection(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_reflection)
sph3_coef_reflection = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_reflection(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_reflection)
tmp2 = sph2_coef_reflection - sph2_coef_baseline
tmp3 = sph3_coef_reflection - sph3_coef_baseline
print("fmm_m2m_reflection:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
sph2_coef_reflection3 = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_reflection3(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_reflection3)
sph3_coef_reflection3 = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_reflection3(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_reflection3)
tmp2 = sph2_coef_reflection3 - sph2_coef_baseline
tmp3 = sph3_coef_reflection3 - sph3_coef_baseline
print("fmm_m2m_reflection3:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
sph2_coef_test_mat = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_test_mat(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_test_mat)
sph3_coef_test_mat = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_test_mat(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_test_mat)
tmp2 = sph2_coef_test_mat - sph2_coef_baseline
tmp3 = sph3_coef_test_mat - sph3_coef_baseline
print("fmm_m2m_test_mat:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
sph2_coef_use_mat = np.zeros_like(sph1_coef)
sph2_reflect_mat, sph2_ztrans_mat = pcm_fmm.fmm_m2m_get_mat(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm)
pcm_fmm.fmm_m2m_use_mat(sph1_c-sph2_c, sph1_r, sph2_r, p, sph2_reflect_mat, sph2_ztrans_mat, sph1_coef,
                        sph2_coef_use_mat)
# Get full M2M matrix to find spectral norm of m2m operation
N = sph1_coef.shape[0]
mat = np.zeros((N, N))
for i in range(N):
    e = np.zeros(N)
    e[i] = 1
    ans = np.zeros_like(sph1_coef)
    pcm_fmm.fmm_m2m_use_mat(sph1_c-sph2_c, sph1_r, sph2_r, p, sph2_reflect_mat, sph2_ztrans_mat, e,
                        ans)
    mat[i] = ans
print(mat@mat.T)
print("norm:", np.linalg.norm(mat, ord=2))

sph3_coef_use_mat = np.zeros_like(sph1_coef)
sph3_reflect_mat, sph3_ztrans_mat = pcm_fmm.fmm_m2m_get_mat(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm)
pcm_fmm.fmm_m2m_use_mat(sph1_c-sph3_c, sph1_r, sph3_r, p, sph3_reflect_mat, sph3_ztrans_mat, sph1_coef,
                        sph3_coef_use_mat)
tmp2 = sph2_coef_use_mat - sph2_coef_baseline
tmp3 = sph3_coef_use_mat - sph3_coef_baseline
print("fmm_m2m_use_mat:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
sph2_coef_fast = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_fast(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_fast)
sph3_coef_fast = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_fast(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_fast)
tmp2 = sph2_coef_fast - sph2_coef_baseline
tmp3 = sph3_coef_fast - sph3_coef_baseline
print("fmm_m2m_fast:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
print("Time for M2M baseline")
%timeit pcm_fmm.fmm_m2m_baseline(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_baseline)
print("Time for M2M reflection")
%timeit pcm_fmm.fmm_m2m_reflection(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_reflection)
print("Time for M2M reflection3")
%timeit pcm_fmm.fmm_m2m_reflection3(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_reflection3)
print("Time for M2M test_mat")
%timeit pcm_fmm.fmm_m2m_test_mat(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_test_mat)
print("Time for M2M get_mat")
%timeit pcm_fmm.fmm_m2m_get_mat(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm)
print("Time for M2M use_mat")
%timeit pcm_fmm.fmm_m2m_use_mat(sph1_c-sph3_c, sph1_r, sph3_r, p, sph3_reflect_mat, sph3_ztrans_mat, sph1_coef,\
                        sph3_coef_use_mat)
print("Time for M2M fast")
%timeit pcm_fmm.fmm_m2m_fast(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_fast)
print("Time for M2M fast (only along OZ axis)")
%timeit pcm_fmm.fmm_m2m_fast(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_fast)
print("Time for M2M ztranslate")
%timeit pcm_fmm.fmm_m2m_ztranslate(1, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_fast)


fmm_m2m_reflection: 1.2659256448454166e-15 1.3478805539742462e-14
fmm_m2m_reflection3: 1.2659256448454166e-15 1.311682060506621e-14
fmm_m2m_test_mat: 1.2659256448454166e-15 1.3156767787291502e-14
[[ 5.55555556e-01  0.00000000e+00 -2.77983463e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.46913580e-01  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-2.77983463e-01  0.00000000e+00  3.33333333e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.79366203e-43
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   1.79366203e-43  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.79366203e-43]]
norm: 1.0041549944093697
fmm_m2m_use_mat: 1.2659256448454166e-15 1.3156767787291502e-14
fmm_m2m_fast: 1.2659256448454166e-15 5.4227172523851655e-15
Time for M2M baselin

In [4]:
# Check optimized M2L for spheres, order of M is greater than order of L
import numpy as np
from test_pcm_fmm import pcm_fmm
pm = 20
pl = 15
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(pm, pl, ngrid)
sph1_c = np.zeros(3)
sph1_r = 1
sph1_coef = np.ones((pm+1)*(pm+1), dtype=np.float64)
sph2_c = np.array([0, 0, -10], dtype=np.float64)
sph2_r = sph1_r + np.linalg.norm(sph2_c-sph1_c)
sph3_c = np.array([1, 2, 10], dtype=np.float64)
sph3_r = sph1_r + np.linalg.norm(sph3_c-sph1_c)
sph2_coef_baseline = np.zeros((pl+1)*(pl+1), dtype=np.float64)
pcm_fmm.fmm_m2l_baseline(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_baseline)
sph3_coef_baseline = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_baseline(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_baseline)
sph2_coef_test_mat = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_test_mat(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_test_mat)
sph3_coef_test_mat = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_test_mat(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_test_mat)
tmp2 = sph2_coef_test_mat - sph2_coef_baseline
tmp3 = sph3_coef_test_mat - sph3_coef_baseline
print("fmm_m2l_test_mat:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
sph2_coef_use_mat = np.zeros_like(sph2_coef_baseline)
sph2_reflect_mat, sph2_ztrans_mat = pcm_fmm.fmm_m2l_get_mat(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales)
pcm_fmm.fmm_m2l_use_mat(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, sph2_reflect_mat, sph2_ztrans_mat, sph1_coef,
                        sph2_coef_use_mat)
sph3_coef_use_mat = np.zeros_like(sph2_coef_baseline)
sph3_reflect_mat, sph3_ztrans_mat = pcm_fmm.fmm_m2l_get_mat(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales)
pcm_fmm.fmm_m2l_use_mat(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, sph3_reflect_mat, sph3_ztrans_mat, sph1_coef,
                        sph3_coef_use_mat)
tmp2 = sph2_coef_use_mat - sph2_coef_baseline
tmp3 = sph3_coef_use_mat - sph3_coef_baseline
print("fmm_m2l_use_mat:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
sph2_coef_fast = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_fast(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_fast)
sph3_coef_fast = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_fast(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_fast)
tmp2 = sph2_coef_fast - sph2_coef_baseline
tmp3 = sph3_coef_fast - sph3_coef_baseline
print("fmm_m2l_fast:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
print("Time for M2L baseline")
%timeit pcm_fmm.fmm_m2l_baseline(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_baseline)
print("Time for M2L test_mat")
%timeit pcm_fmm.fmm_m2l_test_mat(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_test_mat)
print("Time for M2L get_mat")
%timeit pcm_fmm.fmm_m2l_get_mat(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales)
print("Time for M2L use_mat")
%timeit pcm_fmm.fmm_m2l_use_mat(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, sph3_reflect_mat, sph3_ztrans_mat,\
                                sph1_coef, sph3_coef_use_mat)
print("Time for M2L fast")
%timeit pcm_fmm.fmm_m2l_fast(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_fast)
print("Time for M2L fast (only along OZ axis)")
%timeit pcm_fmm.fmm_m2l_fast(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_fast)
print("Time for M2L ztranslate")
%timeit pcm_fmm.fmm_m2l_ztranslate(1, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_fast)

fmm_m2l_test_mat: 3.604281297940481e-16 4.38592831772872e-15
fmm_m2l_use_mat: 3.604281297940481e-16 4.38592831772872e-15
fmm_m2l_fast: 3.604281297940481e-16 2.2006153186012927e-15
Time for M2L baseline
1.36 ms ± 4.32 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Time for M2L test_mat
217 µs ± 1.98 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Time for M2L get_mat
202 µs ± 4.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Time for M2L use_mat
20.8 µs ± 360 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for M2L fast
243 µs ± 1.85 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Time for M2L fast (only along OZ axis)
31 µs ± 146 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for M2L ztranslate
30.1 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
# Check optimized M2L for spheres, order of L is greater than order of M
import numpy as np
from test_pcm_fmm import pcm_fmm
pm = 15
pl = 20
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(pm, pl, ngrid)
sph1_c = np.zeros(3)
sph1_r = 1
sph1_coef = np.ones((pm+1)*(pm+1), dtype=np.float64)
sph2_c = np.array([0, 0, -10], dtype=np.float64)
sph2_r = sph1_r + np.linalg.norm(sph2_c-sph1_c)
sph3_c = np.array([1, 2, 10], dtype=np.float64)
sph3_r = sph1_r + np.linalg.norm(sph3_c-sph1_c)
sph2_coef_baseline = np.zeros((pl+1)*(pl+1), dtype=np.float64)
pcm_fmm.fmm_m2l_baseline(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_baseline)
sph3_coef_baseline = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_baseline(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_baseline)
sph2_coef_test_mat = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_test_mat(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_test_mat)
sph3_coef_test_mat = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_test_mat(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_test_mat)
tmp2 = sph2_coef_test_mat - sph2_coef_baseline
tmp3 = sph3_coef_test_mat - sph3_coef_baseline
print("fmm_m2l_test_mat:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
sph2_coef_use_mat = np.zeros_like(sph2_coef_baseline)
sph2_reflect_mat, sph2_ztrans_mat = pcm_fmm.fmm_m2l_get_mat(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales)
pcm_fmm.fmm_m2l_use_mat(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, sph2_reflect_mat, sph2_ztrans_mat, sph1_coef,
                        sph2_coef_use_mat)
sph3_coef_use_mat = np.zeros_like(sph2_coef_baseline)
sph3_reflect_mat, sph3_ztrans_mat = pcm_fmm.fmm_m2l_get_mat(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales)
pcm_fmm.fmm_m2l_use_mat(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, sph3_reflect_mat, sph3_ztrans_mat, sph1_coef,
                        sph3_coef_use_mat)
tmp2 = sph2_coef_use_mat - sph2_coef_baseline
tmp3 = sph3_coef_use_mat - sph3_coef_baseline
print("fmm_m2l_use_mat:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
sph2_coef_fast = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_fast(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_fast)
sph3_coef_fast = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_fast(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_fast)
tmp2 = sph2_coef_fast - sph2_coef_baseline
tmp3 = sph3_coef_fast - sph3_coef_baseline
print("fmm_m2l_fast:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
print("Time for M2L baseline")
%timeit pcm_fmm.fmm_m2l_baseline(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_baseline)
print("Time for M2L test_mat")
%timeit pcm_fmm.fmm_m2l_test_mat(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_test_mat)
print("Time for M2L get_mat")
%timeit pcm_fmm.fmm_m2l_get_mat(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales)
print("Time for M2L use_mat")
%timeit pcm_fmm.fmm_m2l_use_mat(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, sph3_reflect_mat, sph3_ztrans_mat,\
                                sph1_coef, sph3_coef_use_mat)
print("Time for M2L fast")
%timeit pcm_fmm.fmm_m2l_fast(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_fast)
print("Time for M2L fast (only along OZ axis)")
%timeit pcm_fmm.fmm_m2l_fast(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_fast)
print("Time for M2L ztranslate")
%timeit pcm_fmm.fmm_m2l_ztranslate(1, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_fast)

fmm_m2l_test_mat: 6.098712106477445e-16 5.8528280392328285e-15
fmm_m2l_use_mat: 6.098712106477445e-16 5.8528280392328285e-15
fmm_m2l_fast: 6.098712106477445e-16 2.6319900889926794e-15
Time for M2L baseline
1.34 ms ± 11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Time for M2L test_mat
215 µs ± 769 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Time for M2L get_mat
199 µs ± 2.08 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for M2L use_mat
20.8 µs ± 311 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for M2L fast
241 µs ± 1.77 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Time for M2L fast (only along OZ axis)
31.9 µs ± 300 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for M2L ztranslate
31.2 µs ± 42.8 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
# Check optimized L2L for spheres
import numpy as np
from test_pcm_fmm import pcm_fmm
p = 20
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
vscalesl = vscales[:(p+1)*(p+1)]
sph1_c = np.zeros(3)
sph1_r = 1
sph1_coef = np.ones((p+1)*(p+1), dtype=np.float64)
sph2_c = np.array([0, 0, 10], dtype=np.float64)
sph2_r = sph1_r + np.linalg.norm(sph2_c-sph1_c)
sph3_c = np.array([1, 2, 10], dtype=np.float64)
sph3_r = sph1_r + np.linalg.norm(sph3_c-sph1_c)
sph2_coef_baseline = np.zeros((p+1)*(p+1), dtype=np.float64)
pcm_fmm.fmm_l2l_baseline(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesl, sph1_coef, sph2_coef_baseline)
sph3_coef_baseline = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_l2l_baseline(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesl, sph1_coef, sph3_coef_baseline)
sph2_coef_test_mat = np.zeros_like(sph1_coef)
pcm_fmm.fmm_l2l_test_mat(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesl, sph1_coef, sph2_coef_test_mat)
sph3_coef_test_mat = np.zeros_like(sph1_coef)
pcm_fmm.fmm_l2l_test_mat(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesl, sph1_coef, sph3_coef_test_mat)
tmp2 = sph2_coef_test_mat - sph2_coef_baseline
tmp3 = sph3_coef_test_mat - sph3_coef_baseline
print("fmm_l2l_test_mat:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
sph2_coef_use_mat = np.zeros_like(sph1_coef)
sph2_reflect_mat, sph2_ztrans_mat = pcm_fmm.fmm_l2l_get_mat(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesl)
pcm_fmm.fmm_l2l_use_mat(sph1_c-sph2_c, sph1_r, sph2_r, p, sph2_reflect_mat, sph2_ztrans_mat, sph1_coef,
                        sph2_coef_use_mat)
sph3_coef_use_mat = np.zeros_like(sph1_coef)
sph3_reflect_mat, sph3_ztrans_mat = pcm_fmm.fmm_l2l_get_mat(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesl)
pcm_fmm.fmm_l2l_use_mat(sph1_c-sph3_c, sph1_r, sph3_r, p, sph3_reflect_mat, sph3_ztrans_mat, sph1_coef,
                        sph3_coef_use_mat)
tmp2 = sph2_coef_use_mat - sph2_coef_baseline
tmp3 = sph3_coef_use_mat - sph3_coef_baseline
print("fmm_l2l_use_mat:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
sph2_coef_fast = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_l2l_fast(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesl, sph1_coef, sph2_coef_fast)
sph3_coef_fast = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_l2l_fast(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesl, sph1_coef, sph3_coef_fast)
tmp2 = sph2_coef_fast - sph2_coef_baseline
tmp3 = sph3_coef_fast - sph3_coef_baseline
print("fmm_l2l_fast:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
print("Time for L2L baseline")
%timeit pcm_fmm.fmm_l2l_baseline(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesl, sph1_coef, sph3_coef_baseline)
print("Time for L2L test_mat")
%timeit pcm_fmm.fmm_l2l_test_mat(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesl, sph1_coef, sph3_coef_test_mat)
print("Time for L2L get_mat")
%timeit pcm_fmm.fmm_l2l_get_mat(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm)
print("Time for L2L use_mat")
%timeit pcm_fmm.fmm_l2l_use_mat(sph1_c-sph3_c, sph1_r, sph3_r, p, sph3_reflect_mat, sph3_ztrans_mat, sph1_coef,\
                        sph3_coef_use_mat)
print("Time for L2L fast")
%timeit pcm_fmm.fmm_l2l_fast(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesl, sph1_coef, sph3_coef_fast)
print("Time for L2L fast (only along OZ axis)")
%timeit pcm_fmm.fmm_l2l_fast(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesl, sph1_coef, sph2_coef_fast)
print("Time for L2L ztranslate")
%timeit pcm_fmm.fmm_l2l_ztranslate(1, sph1_r, sph2_r, p, vscalesl, sph1_coef, sph2_coef_fast)

fmm_l2l_test_mat: 1.9197264029805683e-16 6.097363469313388e-15
fmm_l2l_use_mat: 1.9197264029805683e-16 6.097363469313388e-15
fmm_l2l_fast: 1.9197264029805683e-16 9.355749368456677e-16
Time for L2L baseline
448 µs ± 1.85 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Time for L2L test_mat
214 µs ± 3.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Time for L2L get_mat
200 µs ± 913 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Time for L2L use_mat
20.4 µs ± 100 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for L2L fast
333 µs ± 2.11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Time for L2L fast (only along OZ axis)
27.7 µs ± 218 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for L2L ztranslate
26.9 µs ± 588 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
# Check optimized tree M2M
import numpy as np
from time import time
from test_pcm_fmm import pcm_fmm
nsph = 1000
p = 6
nbasis = (p+1)**2
ngrid = 110
csph = np.random.randn(3, nsph)
rsph = 0.1*np.ones(nsph)
ui = np.ones((ngrid, nsph))
coef_sph = np.random.randn(nbasis, nsph)
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
vscalesm = vscales[:(p+1)**2]
ind = np.arange(1, nsph+1, dtype=np.int32)
cluster, children, parent, cnode, rnode, snode = pcm_fmm.btree_init(csph, rsph, ind)
height = pcm_fmm.tree_get_height(parent)
n2 = pcm_fmm.tree_improve_get_size(children, height)
cluster2, children2, parent2, cnode2, rnode2 = pcm_fmm.tree_improve(n2, height, cluster, children,
        parent, cnode, rnode)
m2m_reflect_mat, m2m_ztrans_mat = pcm_fmm.tree_m2m_get_mat(children2, cnode2, rnode2, p, vscalesm)
coef_node_use_mat = pcm_fmm.tree_m2m_use_mat(p, coef_sph, ind, cluster2, children2, cnode2, rnode2,
                                             m2m_reflect_mat, m2m_ztrans_mat)
coef_node = pcm_fmm.tree_m2m_fast(p, vscalesm, coef_sph, ind, cluster2, children2, cnode2, rnode2)
print("tree_m2m_use_mat:", np.linalg.norm(coef_node-coef_node_use_mat) / np.linalg.norm(coef_node))
print("Time for tree M2M fast:")
%timeit pcm_fmm.tree_m2m_fast(p, vscalesm, coef_sph, ind, cluster2, children2, cnode2, rnode2)
print("Time for tree M2M get_mat:")
%timeit pcm_fmm.tree_m2m_get_mat(children2, cnode2, rnode2, p, vscalesm)
print("Time for tree M2M use_mat:")
%timeit pcm_fmm.tree_m2m_use_mat(p, coef_sph, ind, cluster2, children2, cnode2, rnode2,\
                                             m2m_reflect_mat, m2m_ztrans_mat)

AttributeError: 'fortran' object has no attribute 'tree_improve_get_size'

In [ ]:
# Check optimized tree L2L
import numpy as np
from time import time
from test_pcm_fmm import pcm_fmm
nsph = 1000
p = 6
nbasis = (p+1)**2
ngrid = 110
csph = np.random.randn(3, nsph)
rsph = 0.1*np.ones(nsph)
ui = np.ones((ngrid, nsph))
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
vscalesl = vscales[:(p+1)**2]
ind = np.arange(1, nsph+1, dtype=np.int32)
cluster, children, parent, cnode, rnode, snode = pcm_fmm.btree_init(csph, rsph, ind)
height = pcm_fmm.tree_get_height(parent)
n2 = pcm_fmm.tree_improve_get_size(children, height)
cluster2, children2, parent2, cnode2, rnode2 = pcm_fmm.tree_improve(n2, height, cluster, children,
        parent, cnode, rnode)
coef_node = np.random.randn(nbasis, n2).copy('F')
coef_node_use_mat = coef_node.copy('F')
l2l_reflect_mat, l2l_ztrans_mat = pcm_fmm.tree_l2l_get_mat(children2, cnode2, rnode2, p, vscalesl)
coef_sph_use_mat = pcm_fmm.tree_l2l_use_mat(p, coef_node_use_mat, ind, cluster2, children2, cnode2, rnode2,
                                             l2l_reflect_mat, l2l_ztrans_mat)
coef_sph = pcm_fmm.tree_l2l_fast(p, vscalesl, coef_node, ind, cluster2, children2, cnode2, rnode2)
print("tree_l2l_use_mat:", np.linalg.norm(coef_sph-coef_sph_use_mat) / np.linalg.norm(coef_sph))
print("Time for tree L2L fast:")
%timeit pcm_fmm.tree_l2l_fast(p, vscalesl, coef_node, ind, cluster2, children2, cnode2, rnode2)
print("Time for tree L2L get_mat:")
%timeit pcm_fmm.tree_l2l_get_mat(children2, cnode2, rnode2, p, vscalesl)
print("Time for tree L2L use_mat:")
%timeit pcm_fmm.tree_l2l_use_mat(p, coef_node_use_mat, ind, cluster2, children2, cnode2, rnode2,\
                                 l2l_reflect_mat, l2l_ztrans_mat)

In [ ]:
# Check optimized tree M2L
import numpy as np
from time import time
from test_pcm_fmm import pcm_fmm
nsph = 1000
pm = 6
pl = 6
nbasism = (pm+1)**2
nbasisl = (pl+1)**2
ngrid = 110
csph = np.random.randn(3, nsph)
rsph = 0.1*np.ones(nsph)
ui = np.ones((ngrid, nsph))
vscales, w, grid, vgrid = pcm_fmm.init_globals(pm, pl, ngrid)
ind = np.arange(1, nsph+1, dtype=np.int32)
cluster, children, parent, cnode, rnode, snode = pcm_fmm.btree_init(csph, rsph, ind)
height = pcm_fmm.tree_get_height(parent)
n2 = pcm_fmm.tree_improve_get_size(children, height)
cluster2, children2, parent2, cnode2, rnode2 = pcm_fmm.tree_improve(n2, height, cluster, children,
        parent, cnode, rnode)
coef_m = np.random.randn(nbasism, n2)
lwork = nsph*400
iwork = np.zeros(1, dtype=np.int32)
jwork = np.zeros(1, dtype=np.int32)
work = np.zeros((3,lwork), dtype=np.int32, order='F')
nnfar, nfar, nnnear, nnear = pcm_fmm.tree_get_farnear_work(children2, cnode2, rnode2, iwork, jwork, work)
#print(nnfar, nnnear)
#print(n2)
if(iwork[0] != jwork[0]+1):
    print("LWORK IS TOO SMALL")
else:
    sfar, far, snear, near = pcm_fmm.tree_get_farnear(jwork[0], work, nnfar, nfar, nnnear, nnear)
coef_l = pcm_fmm.tree_m2l_fast(cnode2, rnode2, sfar, far, pm, pl, vscales, coef_m)
m2l_reflect_mat, m2l_ztrans_mat = pcm_fmm.tree_m2l_get_mat(cnode2, rnode2, sfar, far, pm, pl, vscales)
coef_l_use_mat = pcm_fmm.tree_m2l_use_mat(cnode2, rnode2, sfar, far, pm, pl, m2l_reflect_mat,\
                                          m2l_ztrans_mat, coef_m)
print("tree_m2l_use_mat:", np.linalg.norm(coef_l-coef_l_use_mat) / np.linalg.norm(coef_l))
print("Time for M2L fast")
%timeit pcm_fmm.tree_m2l_fast(cnode2, rnode2, sfar, far, pm, pl, vscales, coef_m)
print("Time for M2L use_mat")
%timeit pcm_fmm.tree_m2l_get_mat(cnode2, rnode2, sfar, far, pm, pl, vscales)
print("Time for M2L use_mat")
%timeit pcm_fmm.tree_m2l_use_mat(cnode2, rnode2, sfar, far, pm, pl, m2l_reflect_mat,\
                                          m2l_ztrans_mat, coef_m)

In [6]:
import numpy as np
from time import time
from pcm_fmm import pcm_fmm
from mydx import mydx
mydx.init()
nbasis, ngrid, nsph = mydx.get_sizes()
csph, rsph = mydx.get_spheres(nsph)
ui = mydx.get_ui(nsph, ngrid)
p = int(nbasis**0.5)-1
print("nsph = {}\np = {}\nngrid = {}".format(nsph, p, ngrid))
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
ind = np.arange(1, nsph+1, dtype=np.int32)
cluster, children, parent, cnode, rnode, snode = pcm_fmm.btree1_init(csph, rsph, ind)
height = pcm_fmm.tree_get_height(parent)
n2 = pcm_fmm.tree_improve_get_size(children, height)
#print(height, n2)
cluster2, children2, parent2, cnode2, rnode2 = pcm_fmm.tree_improve(n2, height, cluster, children,
                                                                    parent, cnode, rnode)
ind3 = np.arange(1, nsph+1, dtype=np.int32)
cluster3, children3, parent3, cnode3, rnode3, snode3 = pcm_fmm.btree2_init(csph, rsph, ind3)
#print(children.shape, children2.shape)
#print(ind)
#print(cluster)
#print(children)
np.random.seed(0)
coef_sph = np.random.randn(nbasis, nsph)

ModuleNotFoundError: No module named 'mydx'

In [ ]:
t0 = time()
coef_ddpcm = mydx.ddpcm_dx(coef_sph)
print("ddPCM: ", time()-t0, " seconds")

In [ ]:
t0 = time()
coef_out = pcm_fmm.pcm_matvec_grid_treecode(csph, rsph, grid, w, vgrid, ui, p, vscales[:(p+1)*(p+1)], ind, \
                                   cluster, children, cnode, rnode, snode, coef_sph)
print("TREECODE: ", time()-t0, " seconds")
print(np.linalg.norm(coef_ddpcm-coef_out), np.linalg.norm(coef_ddpcm))

In [ ]:
t0 = time()
coef_out2 = pcm_fmm.pcm_matvec_grid_treecode2(csph, rsph, grid, w, vgrid, ui, p, p, vscales, ind, \
                                   cluster, children, cnode, rnode, coef_sph)
print("TREECODE2: ", time()-t0, " seconds")
print(np.linalg.norm(coef_ddpcm-coef_out2), np.linalg.norm(coef_ddpcm))

In [ ]:
lwork = nsph*400
iwork = np.zeros(1, dtype=np.int32)
jwork = np.zeros(1, dtype=np.int32)
work = np.zeros((3,lwork), dtype=np.int32, order='F')
nnfar, nfar, nnnear, nnear = pcm_fmm.tree_get_farnear_work(children, cnode, rnode, iwork, jwork, work)
print(nnfar, nnnear)
if(iwork[0] != jwork[0]+1):
    print("LWORK IS TOO SMALL")
else:
    sfar, far, snear, near = pcm_fmm.tree_get_farnear(jwork[0], work, nnfar, nfar, nnnear, nnear)
    #print(nnnear, nnear, snear, near)
    t0 = time()
    coef_out_fmm = pcm_fmm.pcm_matvec_grid_fmm_baseline(csph, rsph, grid, w, vgrid, ui, p, p, vscales,
                                                        ind, cluster, children, cnode, rnode, sfar, far,
                                                        snear, near, coef_sph)
    print("FMM baseline: ", time()-t0, " seconds",
          np.linalg.norm(coef_ddpcm-coef_out_fmm)/np.linalg.norm(coef_ddpcm))
    t0 = time()
    coef_out_fmm = pcm_fmm.pcm_matvec_grid_fmm_fast(csph, rsph, grid, w, vgrid, ui, p, p, vscales,
                                                        ind, cluster, children, cnode, rnode, sfar, far,
                                                        snear, near, coef_sph)
    print("FMM fast: ", time()-t0, " seconds",
          np.linalg.norm(coef_ddpcm-coef_out_fmm)/np.linalg.norm(coef_ddpcm))
    #print(coef_out_fmm)
    #print(coef_out_fmm-coef_out)
    #print(coef_out)

In [ ]:
lwork = nsph*400
iwork = np.zeros(1, dtype=np.int32)
jwork = np.zeros(1, dtype=np.int32)
work = np.zeros((3,lwork), dtype=np.int32, order='F')
nnfar, nfar, nnnear, nnear = pcm_fmm.tree_get_farnear_work(children2, cnode2, rnode2, iwork, jwork, work)
print(nnfar, nnnear)
if(iwork[0] != jwork[0]+1):
    print("LWORK IS TOO SMALL")
else:
    sfar, far, snear, near = pcm_fmm.tree_get_farnear(jwork[0], work, nnfar, nfar, nnnear, nnear)
    #print(nnnear, nnear, snear, near)
    #print(nnfar, nfar, sfar, far)
    t0 = time()
    coef_out_fmm = pcm_fmm.pcm_matvec_grid_fmm_fast(csph, rsph, grid, w, vgrid, ui, p, p, vscales,
                                                        ind, cluster2, children2, cnode2, rnode2, sfar, far,
                                                        snear, near, coef_sph)
    print("FMM fast (improved tree): ", time()-t0, " seconds",
          np.linalg.norm(coef_ddpcm-coef_out_fmm)/np.linalg.norm(coef_ddpcm))
    #print(coef_out_fmm)
    #print(coef_out_fmm-coef_out)
    #print(coef_out)

In [ ]:
lwork = nsph*400
iwork = np.zeros(1, dtype=np.int32)
jwork = np.zeros(1, dtype=np.int32)
work = np.zeros((3,lwork), dtype=np.int32, order='F')
nnfar, nfar, nnnear, nnear = pcm_fmm.tree_get_farnear_work(children3, cnode3, rnode3, iwork, jwork, work)
print(nnfar, nnnear)
if(iwork[0] != jwork[0]+1):
    print("LWORK IS TOO SMALL")
else:
    sfar, far, snear, near = pcm_fmm.tree_get_farnear(jwork[0], work, nnfar, nfar, nnnear, nnear)
    #print(nnnear, nnear, snear, near)
    #print(nnfar, nfar, sfar, far)
    t0 = time()
    coef_out_fmm = pcm_fmm.pcm_matvec_grid_fmm_fast(csph, rsph, grid, w, vgrid, ui, p, p, vscales,
                                                        ind3, cluster3, children3, cnode3, rnode3, sfar, far,
                                                        snear, near, coef_sph)
    print("FMM fast (balanced tree): ", time()-t0, " seconds",
          np.linalg.norm(coef_ddpcm-coef_out_fmm)/np.linalg.norm(coef_ddpcm))
    #print(coef_out_fmm)
    #print(coef_out_fmm-coef_out)
    #print(coef_out)

In [ ]:
print(height, pcm_fmm.tree_get_height(parent3))